This script will export rasters which calculate the om/com from gabam and us comparted to mtbs and nbac

In [1]:
import os
import ee
import numpy as np
from geeml.extract import extractor
import pandas as pd
import geemap
import os
from google.cloud import storage
from google.cloud import client
# import geemap
# Authenticate GEE
# ee.Authenticate()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"

service_account = 'gee-serdp-upload@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")
ee.Initialize(credentials)
# Initialize GEE with high-volume end-point
# ee.Initialize(opt_url='https=#earthengine-highvolume.googleapis.com')
ee.Initialize()


os.environ["GCLOUD_PROJECT"] = "gee-serdp-upload"
storage_client = storage.Client()
# bucket_name = 'smp-scratch/mtbs_1985'
bucket_name = 'smp-scratch'

bucket = storage_client.bucket(bucket_name)

In [3]:
#year of interest
year = 2015

lfdb = ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_ca_1985")
aoi2 = ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_ca_west")
t3 = ee.ImageCollection("users/spotter/fire_cnn/mtbs_predictions/all_ak_ca_dnbr_128_56")
table = ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_ca_grd_west_largest2")
viirs = ee.Image("users/spotter/fire_cnn/ak_ca_viirs_2014")
ak = ee.FeatureCollection("users/spotter/alaska")
sk = ee.FeatureCollection("users/spotter/SK_boundary")
buffer = ee.FeatureCollection("users/spotter/fire_cnn/raw/2001_2023_buff").filter(ee.Filter.eq('Year',  2015))   
water = ee.ImageCollection("JRC/GSW1_4/YearlyHistory")
grd = ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_ca_grd_west_largest2")
territory22 = ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_ca_grd_west_largest2")

water = water.filterDate('2015-01-01', '2015-12-31').max().select('waterClass').clip(table.union())
water = water.updateMask(water.eq(3))

#modis land cover for now
lc = ee.ImageCollection("MODIS/061/MCD12Q1")

#read in gabam which is a 30m random forest model
gabam = ee.ImageCollection("projects/sat-io/open-datasets/GABAM")

gabam = gabam.filterDate('2015-01-01', '2015-12-31').mean()

#clip lc
lc = lc.filterDate('2015-01-01', '2015-12-31').max().clip(table.union())
lc = lc.select('LC_Type1')


#union to a single feature for the table
aoi = table.union()
dnbr_ind = ee.ImageCollection("users/spotter/fire_cnn/all_dnbr_individual")
dnbr_ind = dnbr_ind.max().clip(aoi)

#geometry is now aoi
geometry = aoi


def to_short(image): 
    image1 = image.select('pre_7_NBR').cast({'pre_7_NBR': 'short'})
    image2 = image.select('pos_7_NBR').cast({'pos_7_NBR': 'short'})
    image3 = image.select('pre_8_NBR').cast({'pre_8_NBR': 'short'})
    image4 = image.select('pos_8_NBR').cast({'pos_8_NBR': 'short'})

    image = image.addBands(image1).addBands(image2).addBands(image3).addBands(image4)
    return image

 
#read in viirs
viirs = ee.Image("users/spotter/fire_cnn/VIIRS/" + str(2015))
viirs = viirs.clip(territory22).select(['b1'], ['constant'])

#get modis active fire and merge with viirs
Terra_Fire = ee.ImageCollection("MODIS/061/MOD14A1").filterBounds(aoi)
Aqua_Fire = ee.ImageCollection("MODIS/061/MYD14A1").filterBounds(aoi)

#modis info
# modis1kmscale = Aqua_Fire.first().projection().nominalScale().getInfo()
# modis1kmTransform = Aqua_Fire.first().projection().getInfo().transform

mod_fire = Terra_Fire.merge(Aqua_Fire).filterBounds(aoi).filterDate(str(year) + '-01-01', str(year) + '-12-31')

def add_bands(image):
    image = image.addBands(ee.Image(ee.Date(image.get('system:time_start')).getRelative('day','year').add(1)).clamp(1,366)).updateMask(image.select('FireMask').gte(7))
    return image.updateMask(image.select('constant').gt(60))
    
    
mod_fire =  mod_fire.map(add_bands)

mod_fire = mod_fire.select('constant').max().clip(aoi)



#merge modis and viirs
mod_viirs = ee.ImageCollection([viirs, mod_fire]).max()

#now buffer the buffered 2015 fire polygons, turn it to an image and merge with modis and viirs
def to_year(f):
    return f.set({'Year': year})


buffer = buffer.map(to_year)

buff_img = buffer.reduceToImage(
  properties= ['Year'],
  reducer= ee.Reducer.max()).clip(aoi).select(['max'], ['constant'])



#merge all
final = ee.ImageCollection([mod_viirs, buff_img]).max().updateMask(lc.neq(12).And(lc.neq(16)))

# read in the predictions and mask them with the areas
predictions = ee.ImageCollection('projects/gee-serdp-upload/assets/cnn_mapping/ak_nbac_2d_test_128_32_proj_neg').max().clip(aoi)
#predictions = ee.ImageCollection("users/spotter/fire_cnn/mtbs_predictions/all_ak_ca_dnbr_l8_crop_mtn_small").max().clip(aoi)

#threshold at 0.5 probability
thresholded = predictions.select('prediction').updateMask(final).updateMask(predictions.select('prediction').gte(0.5))

#water mask
thresholded = thresholded.updateMask(water.unmask().Not())

#remove areas from gabam where I didn't have imagery
gabam = gabam.updateMask(predictions)


gabam = gabam.updateMask(lc.neq(12).And(lc.neq(16)))


#large fire databases which is mtbs and nbac
lfdb = ee.FeatureCollection('users/spotter/fire_cnn/raw/ak_ca_1985')

lfdb = lfdb.filter(ee.Filter.eq("Year", year))


#make image of fire databases
lfdb_img = lfdb.reduceToImage(
  properties= ['Year'],
  reducer= ee.Reducer.max()).clip(aoi).select(['max'], ['constant'])

#water mask
lfdb_img = lfdb_img.updateMask(water.unmask().Not())

#get intersection of us and mtbs
us_int = lfdb_img.updateMask(predictions.gte(0.5)).select(['constant'], ['us_int'])

#get omission of us and mtbs (mtbs has fire we do not)
us_om = lfdb_img.updateMask(thresholded.unmask().Not()).select(['constant'], ['us_om'])

#get comission of us and mtbs  (we have fire but mtbs does not)
us_com = thresholded.updateMask(lfdb_img.unmask().Not()).select(['prediction'], ['us_com'])

#get intersection of gabam and mtbs
gabam_int = lfdb_img.updateMask(gabam).select(['constant'], ['gabam_int'])

#get omission of gabam and mtbs
gabam_om = lfdb_img.updateMask(gabam.unmask().Not()).select(['constant'], ['gabam_om'])

#get comission of gabam and mtbs
gabam_com = gabam.updateMask(lfdb_img.unmask().Not()).select(['b1'], ['gabam_com'])

#no loop through the sub grid cells and get the area of each
# all_territories = ee.List(territory22.distinct(["ID"]).aggregate_array("ID")).getInfo()

# all_territories = [4306]
all_territories = [31, 30, 29, 28,
                  48, 47, 46, 45,
                  69, 68, 67, 66,
                  92, 91, 90, 89] #central ak

# all_territories = [67] #central ak
final = []
for this_id in all_territories:
    
    geometry = grd.filter(ee.Filter.eq('ID',  this_id))

    #now get areas and convert to feature collection and send out to csv
    #now get the areas
    pa = ee.Image.pixelArea()

    #areas are in m2

    img_area = pa.updateMask(lfdb_img).reduceRegion(
        reducer= ee.Reducer.sum(),
        geometry= geometry,
        scale= 30,
        tileScale= 16,
        crs= 'EPSG:3413',
        # maxPixels= 1e9,
        bestEffort=False,
        maxPixels= 1e13
    ).get('area')

    us_int_area = pa.updateMask(us_int).reduceRegion(
        reducer= ee.Reducer.sum(),
        geometry= geometry,
        scale= 30,
        tileScale= 16,
        crs= 'EPSG:3413',
        # maxPixels= 1e9,
        bestEffort=False,
        maxPixels= 1e13
    ).get('area')

    us_om_area = pa.updateMask(us_om).reduceRegion(
        reducer= ee.Reducer.sum(),
        geometry= geometry,
        scale= 30,
        tileScale= 16,
        crs= 'EPSG:3413',
        # maxPixels= 1e9,
        bestEffort=False,
        maxPixels= 1e13
      ).get('area')

    us_com_area = pa.updateMask(us_com).reduceRegion(
        reducer= ee.Reducer.sum(),
        geometry= geometry,
        scale= 30,
        tileScale= 16,
        crs= 'EPSG:3413',
        # maxPixels= 1e9,
        bestEffort=False,
        maxPixels= 1e13
    ).get('area')

    gabam_int_area = pa.updateMask(gabam_int).reduceRegion(
        reducer= ee.Reducer.sum(),
        geometry= geometry,
        scale= 30,
        tileScale= 16,
        crs= 'EPSG:3413',
        # maxPixels= 1e9,
        bestEffort=False,
        maxPixels= 1e13
    ).get('area')

    gabam_om_area = pa.updateMask(gabam_om).reduceRegion(
        reducer= ee.Reducer.sum(),
        geometry= geometry,
        scale= 30,
        tileScale= 16,
        crs = 'EPSG:3413',
        # maxPixels= 1e9,
        bestEffort=False,
        maxPixels= 1e13
      ).get('area')

    gabam_com_area = pa.updateMask(gabam_com).reduceRegion(
        reducer= ee.Reducer.sum(),
        geometry= geometry,
        scale= 30,
        tileScale= 16,
        crs= 'EPSG:3413',
        # maxPixels= 1e9,
        bestEffort=False,
        maxPixels= 1e13
    ).get('area')
    
    
    us_int_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Us_Int', 'Area': us_int_area})
    us_om_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Us_Om', 'Area': us_om_area})
    us_com_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Us_Com', 'Area': us_com_area})
    gabam_int_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Gabam_Int', 'Area': gabam_int_area})
    gabam_om_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Gabam_Om', 'Area': gabam_om_area})
    gabam_com_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Gabam_Com', 'Area': gabam_com_area})
    fire_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Fire_Area', 'Area': img_area})

    final_sub = ee.FeatureCollection([us_int_feat, us_om_feat, us_com_feat, gabam_int_feat, gabam_om_feat, gabam_com_feat, fire_feat])
    # final.append(final_sub)
    


    print('Downloading ' +  'om_com_' + str(year) + '_' + str(this_id))
    task = ee.batch.Export.table.toCloudStorage(collection = final_sub, 
                                description = 'om_com_' + str(year) + '_' + str(this_id),
                                bucket =  'smp-scratch', 
                                fileFormat = 'CSV')

    task.start()




In [6]:
't'

't'

In [7]:
final_sub.getInfo()

{'type': 'FeatureCollection',
 'columns': {'Area': 'Float',
  'Class': 'String',
  'ID': 'Integer',
  'Year': 'Integer',
  'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': None,
   'id': '0',
   'properties': {'Area': 333842499.6593896,
    'Class': 'Us_Int',
    'ID': 4306,
    'Year': 2014}},
  {'type': 'Feature',
   'geometry': None,
   'id': '1',
   'properties': {'Area': 194943733.94898078,
    'Class': 'Us_Om',
    'ID': 4306,
    'Year': 2014}},
  {'type': 'Feature',
   'geometry': None,
   'id': '2',
   'properties': {'Area': 7376342.274112241,
    'Class': 'Us_Com',
    'ID': 4306,
    'Year': 2014}},
  {'type': 'Feature',
   'geometry': None,
   'id': '3',
   'properties': {'Area': 291982678.00173384,
    'Class': 'Gabam_Int',
    'ID': 4306,
    'Year': 2014}},
  {'type': 'Feature',
   'geometry': None,
   'id': '4',
   'properties': {'Area': 82288953.22134076,
    'Class': 'Gabam_Om',
    'ID': 4306,
    'Year': 2014}},
  {'type': 'Feature',
   'g